<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/t81_558_class_11_4_hf_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks
**Module 11: Natural Language Processing with Hugging Face**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

* Part 11.1: Introduction to Hugging Face [[Video]](https://www.youtube.com/watch?v=PzuL84ksRuE&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_11_1_hf.ipynb)
* Part 11.2: Hugging Face in Python [[Video]](https://www.youtube.com/watch?v=tkGIF4CFoV4&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_11_2_py_huggingface.ipynb)
* Part 11.3: Hugging Face Tokenizers [[Video]](https://www.youtube.com/watch?v=Cz2nvfK28eI&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_11_3_tokenizers.ipynb)
* **Part 11.4: Hugging Face Datasets** [[Video]](https://www.youtube.com/watch?v=yLlCZLzE2XU&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_11_4_hf_datasets.ipynb)
* Part 11.5: Training Hugging Face Models [[Video]](https://www.youtube.com/watch?v=7YZOik5S3vs&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_11_5_hf_train.ipynb)


# Google CoLab Instructions

The following code checks if Google CoLab is running.

In [ ]:
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Part 11.4: Hugging Face Datasets

The Hugging Face hub includes data sets useful for natural language processing (NLP). The Hugging Face library provides functions that allow you to navigate and obtain these data sets. When we access Hugging Face data sets, the data is in a format specific to Hugging Face. In this part, we will explore this format and see how to convert it to Pandas or TensorFlow data.

We begin by installing Hugging Face if needed. It is also essential to install Hugging Face datasets.



In [ ]:
# HIDE OUTPUT
!pip install transformers
!pip install transformers[sentencepiece]
!pip install datasets

We begin by querying Hugging Face to obtain the total count and names of the data sets. This code obtains the total count and the names of the first five datasets.

In [ ]:
from datasets import list_datasets

all_datasets = list_datasets()

print(f"Hugging Face hub currently contains {len(all_datasets)}")
print(f"datasets. The first 5 are:")
print("\n".join(all_datasets[:10]))


We begin by loading the emotion data set from the Hugging Face hub. [Emotion](https://huggingface.co/datasets/emotion) is a dataset of English Twitter messages with six basic emotions: anger, fear, joy, love, sadness, and surprise. [[Cite:saravia2018carer]](https://paperswithcode.com/paper/carer-contextualized-affect-representations) The following code loads the emotion data set from the Hugging Face hub.

In [ ]:
from datasets import load_dataset

emotions = load_dataset("emotion")


A quick scan of the downloaded data set reveals its structure. In this case, Hugging Face already separated the data into training, validation, and test data sets. The training set consists of 16,000 observations, while the test and validation sets contain 2,000 observations. The dataset is a Python dictionary that includes a Dataset object for each of these three divisions. The datasets only contain two columns, the text and the emotion label for each text sample.

In [ ]:
emotions

You can see a single observation from the training data set here. This observation includes both the text sample and the assigned emotion label. The label is a numeric index representing the assigned emotion.

In [ ]:
emotions['train'][2]


We can display the labels in order of their index labels.

In [ ]:
emotions['train'].features


Hugging face can provide these data sets in a variety of formats. The following code receives the emotion data set as a Pandas data frame.

In [ ]:
import pandas as pd

emotions.set_format(type='pandas')
df = emotions["train"][:]
df[:5]


We can use the Pandas "apply" function to add the textual label for each observation.

In [ ]:
def label_it(row):
  return emotions["train"].features["label"].int2str(row)


df['label_name'] = df["label"].apply(label_it)
df[:5]


With the data in Pandas format and textually labeled, we can display a bar chart of the frequency of each of the emotions.

In [ ]:
import matplotlib.pyplot as plt

df["label_name"].value_counts(ascending=True).plot.barh()
plt.show()


Finally, we utilize Hugging Face tokenizers and data sets together. The following code tokenizes the entire emotion data set. You can see below that the code has transformed the training set into subword tokens that are now ready to be used in conjunction with a transformer for either inference or training.

In [ ]:
from transformers import AutoTokenizer


def tokenize(rows):
    return tokenizer(rows['text'], padding=True, truncation=True)


model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

emotions.set_format(type=None)

encoded = tokenize(emotions["train"][:2])

print("**Input IDs**")
for a in encoded.input_ids:
    print(a)
